In [1]:
import boto3
import pandas as pd
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import recall_score

## Defining the bucket
s3 = boto3.resource('s3')
bucket_name = 'danhtran358-data-445-bucket'
bucket = s3.Bucket(bucket_name)

## Defining the csv file
file_key = 'framingham.csv'

bucket_object = bucket.Object(file_key)
file_object = bucket_object.get()
file_content_stream = file_object.get('Body')

## Reading the csv file
heart = pd.read_csv(file_content_stream)
heart = heart.dropna()
heart.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [2]:
## Defining input and target variables
X = heart[['male', 'age', 'currentSmoker', 'totChol', 'sysBP', 'BMI', 'heartRate', 'glucose']]
Y = heart['TenYearCHD']

# Splitting the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify = Y)

In [5]:
## Tranforming data to 0-1 scale
scaler = MinMaxScaler()

X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns = X_train.columns)

In [6]:
X_train.head()

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,1.0,0.243243,0.0,0.139630,0.196217,0.076641,0.161616,0.084746
1,0.0,0.297297,0.0,0.295688,0.361702,0.146670,0.565657,0.124294
2,1.0,0.837838,1.0,0.320329,0.456265,0.207150,0.282828,0.124294
3,0.0,0.648649,0.0,0.342916,0.082742,0.300196,0.353535,0.076271
4,0.0,0.486486,0.0,0.238193,0.177305,0.174584,0.191919,0.098870


In [7]:
X_test.head()

,male,age,currentSmoker,totChol,sysBP,BMI,heartRate,glucose
0,1.0,0.216216,1.0,0.210526,0.339683,0.278959,0.171429,0.052632
1,1.0,0.756757,1.0,0.269737,0.288889,0.305540,0.171429,0.108187
2,0.0,0.594595,0.0,0.483553,0.463492,0.411304,0.057143,0.076023
3,1.0,0.243243,1.0,0.101974,0.155556,0.123951,0.357143,0.073099
4,0.0,0.837838,1.0,0.680921,0.269841,0.139340,0.142857,0.052632


In [8]:
## LASSO
lasso_cv = LassoCV(cv = 5).fit(X_train, Y_train)
lasso_cv.alpha_

## Building LASSO regression
lasso_md = Lasso(alpha = lasso_cv.alpha_).fit(X_train, Y_train)
lasso_md.coef_

array([ 0.05908459,  0.28558791,  0.04979631,  0.        ,  0.51099164,
        0.        , -0.        ,  0.35164854])

In [10]:
## Select top 5 input variables
X_train_lasso = X_train[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]
X_test_lasso = X_test[['male', 'age', 'currentSmoker', 'sysBP', 'glucose']]

## Logistic regression
logit_md = LogisticRegression().fit(X_train_lasso, Y_train)

## Predicting
import numpy as np
pred1 = logit_md.predict_proba(X_test_lasso)[:,1]
pred1 = np.where(pred1 < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, pred1)

0.8839285714285714

In [11]:
## Random forest classifier
RF = RandomForestClassifier(n_estimators = 500).fit(X_train, Y_train)

## Extract importance
RF.feature_importances_

array([0.02220181, 0.15230791, 0.02194728, 0.16156399, 0.18480154,
       0.17253531, 0.12428932, 0.16035284])

In [14]:
X_train_RF = X_train[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]
X_test_RF = X_test[['age', 'totChol', 'sysBP', 'BMI', 'glucose']]

## Building the random forest
RF = RandomForestClassifier(n_estimators = 500, max_depth = 5).fit(X_train_RF, Y_train)

## Predicting
pred2 = RF.predict_proba(X_test_RF)[:,1]
pred2 = np.where(pred2 < 0.1, 0, 1)

## Computing the recall
recall_score(Y_test, pred2)

0.9017857142857143

In [ ]:
## I would use the random forest to predict